## Example protein design notebook
### Design of a protein minibinder

Recent developments using AI for protein design and engineering have allowed bespoke protein designs to be generated *in silico* to achieve certain specific design criteria.

In this case we are going to design a protein that will bind to the [p53](https://www.uniprot.org/uniprotkb/P04637/entry) tumour suppressor protein at the protein:DNA interface.

This workflow briefly demonstrates the following fundamental protein design steps:
- Predict the structure of a target protein of interest using [Boltz2](https://www.rxrx.ai/boltz-2), an open-source implementation of [AlphaFold3](https://github.com/google-deepmind/alphafold3). This allows us to identify the protein:DNA interface.
- Design an artificial protein binder using [RFdiffusion](https://www.nature.com/articles/s41586-023-06415-8) that will bind to p53 at the protein:DNA interface.
- Generate a sequence using [proteinMPNN](https://www.science.org/doi/10.1126/science.add2187) that will fold into the desired binder structure.
- Analyse the generated sequences and select a preferred sequence.
- Predict the structure of the target protein with the designed binder again to make sure it folds into the correct structure and binds at the desired interface.

This example code could readily be expanded to include:
- Generation of additional designs
- Generation of additional test sequences
- Additional analysis and screening steps to preselect sequences for wet lab screening.
- Generation of expression sequences and constructs.
- Generation of order forms and automation picklists.

In [ ]:
#@title Install dependencies (~2min)

runtime = "GPU(L4 or T4)"

import os
import subprocess

print('Installing dependencies... ', end='')
dependencies = "torch torchvision torchaudio numpy hydra-core pytorch-lightning py3dmol "
dependencies += "rdkit dm-tree requests pandas types-requests einops einx fairscale "
dependencies += "mashumaro modelcif wandb click pyyaml biopython scipy numba gemmi "
dependencies += "scikit-learn chembl_structure_pipeline "
dependencies += "cuequivariance_ops_cu12 cuequivariance_ops_torch_cu12 cuequivariance_torch"

if runtime == "GPU(L4 or T4)":
    precision = '32-true'
else:
    precision = 'bf16-true'

subprocess.run("pip install ipywidgets torch torchvision torchaudio", shell=True)
subprocess.run("git clone https://github.com/jwohlwend/boltz.git", shell=True)
subprocess.run(f"sed -i 's/bf16-mixed/{precision}/g' /content/boltz/src/boltz/main.py", shell=True)
!pip install {dependencies} --quiet
subprocess.run("cd boltz; pip install --no-deps -e .", shell=True)

print('done.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.2/118.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.5/392.5 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
import numpy
import torch
import pathlib
import boltz
import tempfile
import py3Dmol

if not torch.cuda.is_available():
  raise RuntimeError("This notebook requires a GPU to run. Request a GPU from Google Colab by going to Runtime > Change runtime type and selecting 'T4 GPU'")

This notebook will be very slow without a GPU. Request a GPU from Google Colab by going to Runtime > Change runtime type and selecting 'T4 GPU'


### Predict the target protein structure

First we will predict the structure of p53 bound to DNA. In this case we can use the protein and DNA sequences used to solve the crystal structure of p53 bound to DNA, [1TSR](https://www.rcsb.org/structure/1TSR):

In [ ]:
# list of input FASTA strings, labelled as protein or dna as required by Boltz
fasta_list = [">A|protein\nSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNT",
              ">B|dna\nTTTCCTAGACTTGCCCAATTA",
              ">C|dna\nATAATTGGGCAAGTCTAGGAA"]

# make a tempfile to use as input for boltz to avoid having to generate unnecessary files
fasta_text = "\n".join(fasta_list)

with tempfile.NamedTemporaryFile(mode="w", suffix=".fasta", delete=False) as fh:
    fh.write(fasta_text)
    fasta_path = pathlib.Path(fh.name)

# get the filename from the output path to use as an output ID
output_id = fasta_path.name.split(".")[0]

output_dir = pathlib.Path("boltz_output")

!boltz predict {fasta_path} --out_dir {output_dir} --use_msa_server


MSA server enabled: https://api.colabfold.com
MSA server authentication: no credentials provided
Extracting the CCD data to /root/.boltz/mols. This may take a bit of time. You may change the cache directory with the --cache flag.
Checking input data.
Processing 1 inputs with 1 threads.
  0% 0/1 [00:00<?, ?it/s]Generating MSA for /tmp/tmpz_cwhjmm.fasta with 1 protein entities.
Calling MSA server for target tmpz_cwhjmm with 1 sequences
MSA server URL: https://api.colabfold.com
MSA pairing strategy: greedy
No authentication provided for MSA server

  0% 0/150 [00:00<?, ?it/s]
SUBMIT:   0% 0/150 [00:00<?, ?it/s]
COMPLETE:   0% 0/150 [00:00<?, ?it/s]
COMPLETE: 100% 150/150 [00:01<00:00, 77.76it/s] 
100% 1/1 [00:02<00:00,  2.43s/it]
Traceback (most recent call last):
  File "/usr/local/bin/boltz", line 8, in <module>
    sys.exit(cli())
             ^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1442, in __call__
    return self.main(*args, **kwargs)
           ^^

## Compare the prediction to the PDB entry

In this case there is a PDB entry we can use as a reference and we can visualise this here and see that the prediction is quite accurate.

First this is the PDB entry:

In [ ]:
# view the original PDB file:
view = py3Dmol.view(query='pdb:1TSR',style={'cartoon':{}}, width=800, height=600)
view.setStyle({"cartoon": {"color": "spectrum"}})
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now the prediction:

In [ ]:
# this is the location of the output predicted structure
prediction_location = output_dir / f"boltz_results_{output_id}" / "predictions" / output_id / f"{output_id}_model_0.cif"

cif_str = prediction_location.read_text()
view = py3Dmol.view()
view.addModel(cif_str, 'mmcif')
view.setStyle({"cartoon": {"color": "spectrum"}})
view.zoomTo()
view.show()

FileNotFoundError: [Errno 2] No such file or directory: 'boltz_output/boltz_results_tmpz_cwhjmm/predictions/tmpz_cwhjmm/tmpz_cwhjmm_model_0.cif'

## Design a binder

We would like to design an artificial protein that will bind to the DNA-binding interface of p53. From the predicted structure we can identify the DNA binding interface and key residues involved in the protein:DNA interaction. The region around Ser241 seems to be a crucial interaction site with the DNA, so we will designate some 'hotspot' residues: Ser241, Ala276 and Asp281.

On a real project you might want to probe the interface further with, for example, mutagenesis or molecular dynamics data.

I designed a series of 10 binders (each 100-residues) that will bind to this interface using RFdiffusion using the command:

`./scripts/run_inference.py inference.output_prefix=binder_outputs/p53_binder inference.input_pdb=input_pdbs/1tsr.pdb 'contigmap.contigs=[A94-289/0 100-100]' 'ppi.hotspot_res=[A241,A276,A281]' inference.num_designs=10`

I haven't made this part of the notebook for simplicity - RFdiffusion is setup more as a CLI tool and takes a while to run without extra compute resources.

I selected the last of these designs and used proteinMPNN to design sequences that would yield the desired structure (ProteinMPNN is also more of a CLI tool):

`python protein_mpnn_run.py --pdb_path input_files/p53_binder_9.pdb --out_folder output_files --num_seq_per_target 10 --sampling_temp "0.1" --seed 0 --batch_size 1 --model_name v_48_020`

This resulted in 10 binder sequences for this one design (it is possible to design many more binder structures and multiple sequences for each structure):

In [ ]:
binder_sequences = ["SLAALLAAAALAAAAAAAAARQPEADAAILAAVRAAVAALRAAIAAAAAAGVDREAAVAAARAMAAAVEATVARHAAACAPLAPEVAALAAEVAALAAEL",
"SAAAAAAAAAAAAAAAAAAARQPARDAEIQAEVAAAIAAVRAARAAAAAAGVDKAAAVAALKAAVDAVVATVAKHAAATAPLKAEVEAVRAEVDALAAAL",
"DAAAAAAAAAAAAAAAAAAAAAPARAAAIRAEVRAAIAALRELRKRAEEAGVDRAEAVAALAALAEAIAAAAAAHAAATAPLRPEIEAARAEVEALRAEL",
"SAAAAAAAAALAAAAAAAEAEAPARAAAIRAEVAAGIEELEEKEEEYEEEGVDEEEGVEELEEMMEEIVALAERHPEAVAPLKPRIEERRERVEELKEEL",
"SAAAAAAAAAAAAAAAAAEAAAPARCAAIQAEVRAAIAAIEAAIEQAEKEGVDPAAAIAAARAMVDEVRARAERHAACTAPLAPEIDAVAARVAARAAAA",
"DLAALLAAALAAALAAAAAAAAPARDAAILAELRERIAAIRELKAEAERAGVDRAAAVAALAAAMAEARALAAAHPAATAPLAPELDAVAAEVAALVEAL",
"SLAAALAAALAAAAAAAAAAAQPAADAAVRAAVAAAIAALRAARAAAAAAGVDPAAAVAALAAAADAVRAAAAKHPAAAAPLAPEIAAAAAAVAAEAAAL",
"SAAAAAAAAAAAAAAAAAAARQPEREAEIKEKLKKMIEEIKELIKKAKKEGIDKKKAVEEAEKKVEEAKELAEKHPEATEPLKEELEKVKKEVEELKKKL",
"SLLAALAAALAAAAAAAAAAALPARCAAIQEKVRKMIEELKEEIKRAKEEGVDKKEAVERAEKMMEEIVKLAEEHPECVAPLKPEIEKAKKEVEEMKEKL",
"SPLAALLAALAAALAAALAAAQPARDAAIQAALQAAIAATREAIAAAAAAGVDPAAAAAAAAAMVAAARALAARNPAACAPLQPELDAVAAEVDALVAAL"]

## Analyse and select a design

Lets make a multiple sequence alignment from these binder sequences:

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

alignment = MultipleSeqAlignment([SeqRecord(Seq(sequence), id=f"binder{binder_sequences.index(sequence)}") for sequence in binder_sequences])
print(alignment)

We can now identify the sequence with the highest solubility for this example. This can be a useful metric to select binders to test, perhaps coupled with other metrics such as stability.

We are using a very simple measure of solubility, just counting the fraction of hydrophilic residues. For a more sophisticated analysis we could use a deep-learning framework such as [DeepSol](https://academic.oup.com/bioinformatics/article/34/15/2605/4938490).

In [ ]:
def simple_solubility_score(seq):
    hydrophilic_residues = "DEHKR"
    return sum(1 for aa in seq if aa in hydrophilic_residues) / len(seq)

solubility_scores = {}
for sequence in binder_sequences:
  solubility_scores[sequence] = simple_solubility_score(sequence)

print(solubility_scores)

soluble_binder = max(solubility_scores, key=solubility_scores.get)
print(f"The sequence with the highest solubility score is: {soluble_binder}")

In our example case, we are trying to do something quite difficult which is to design a binder to a charged polar site on p53 so I have taken all the sequences for this design and predicted the structure of the complex with p53 to identify one which binds at the desired interface.

The selected binder sequence does not have great solubility but does appear to bind at the right site, including our specified hotspot residues. For a real project it might be preferable to leverage more compute power to design multiple binder structures and multiple sequences per design, then set up a workflow to systematically predict and analyse the bound structures and filter to those with preferred properties.

In [ ]:
p53_and_binder = [fasta_list[0],
              f">B|protein\n{binder_sequences[5]}]"]

# make a tempfile to use as input for boltz to avoid having to generate unnecessary files
p53_and_binder_text = "\n".join(p53_and_binder)

with tempfile.NamedTemporaryFile(mode="w", suffix=".fasta", delete=False) as fh:
    fh.write(p53_and_binder_text)
    fasta_path = pathlib.Path(fh.name)

output_id = fasta_path.name.split(".")[0]

output_dir = pathlib.Path("boltz_output")

!boltz predict {fasta_path} --out_dir {output_dir} --use_msa_server

# this is the location of the output predicted structure
prediction_location = output_dir / f"boltz_results_{output_id}" / "predictions" / output_id / f"{output_id}_model_0.cif"

print("loading prediction: ", prediction_location)

cif_str = prediction_location.read_text()
view = py3Dmol.view()
view.addModel(cif_str, 'mmcif')
view.setStyle({"cartoon": {"color": "spectrum"}})
view.zoomTo()
view.show()